In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import numba
import re
from cytoolz import update_in, dissoc
import requests
import holoviews as hv
import matplotlib.pyplot as plt
import pint
import nd2reader
import tifffile
import scipy.stats
import skimage
import skimage.filters
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2
import simulation

In [ ]:
# u = simulation.ureg
class ObjProxy(object):
    def __init__(self, module_name, attr_name):
        self.__module_name = module_name
        self.__attr_name = attr_name

    def __getattr__(self, name):
        return getattr(getattr(globals()[self.__module_name], self.__attr_name), name)


u = ObjProxy("simulation", "ureg")

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
excitation_bins = np.linspace(300, 850, 200)
excitation_bin_size = (excitation_bins[-1] - excitation_bins[0]) / len(excitation_bins)
emission_bins = np.linspace(300, 850, 300)  # excitation_bins
emission_bin_size = (emission_bins[-1] - emission_bins[0]) / len(emission_bins)

# FP spectra

In [ ]:
spectra_urls = [
    "https://www.fpbase.org/spectra_csv/?q=79,80,158",
    "https://www.fpbase.org/spectra_csv/?q=1451,1450",
    "https://www.fpbase.org/spectra_csv/?q=119,120",
    "https://www.fpbase.org/spectra_csv/?q=121,122",
]
spectra = simulation.import_fpbase_spectra(spectra_urls)

In [ ]:
fp_names = ["background", *spectra.keys()]
fp_excitation_spectra = np.stack(
    [
        np.zeros(len(excitation_bins)),
        *[
            simulation.interpolate_dataframe(spectrum["ex"], excitation_bins).values
            for spectrum in spectra.values()
        ],
    ]
).astype(np.float32)
fp_excitation_spectra = xr.DataArray(
    fp_excitation_spectra,
    coords=dict(fp=fp_names, ex=excitation_bins),
    dims=["fp", "ex"],
)
fp_excitation_spectra = fp_excitation_spectra.fillna(0)
seesaw_amounts = np.linspace(-0.5, 1, len(excitation_bins))[np.newaxis, :]
fp_emission_spectra = (
    np.stack(
        [
            np.zeros((len(emission_bins), len(excitation_bins))),
            *[
                simulation.seesaw_spectrum(
                    simulation.interpolate_dataframe(
                        spectrum["em"], emission_bins
                    ).values[:, np.newaxis],
                    seesaw_amounts,
                )
                for spectrum in spectra.values()
            ],
        ]
    )
    .astype(np.float32)
    .swapaxes(1, 2)
)
fp_emission_spectra = xr.DataArray(
    fp_emission_spectra,
    coords=dict(fp=fp_names, ex=excitation_bins, em=emission_bins),
    dims=["fp", "ex", "em"],
)
fp_emission_spectra = fp_emission_spectra.fillna(0)

In [ ]:
fp_emission_spectra[3].plot()

# Excitation line

In [ ]:
line_img = simulation.draw_excitation_line(
    np.linspace(0.3, 1.3, len(excitation_bins)) * u.mm,
    0.3 * u.um,
    0 * u.um,
    1,
    p_vertical=3,
    p_horizontal=6,
    height_padding_factor=10,
    height_px=100,
    width_px=6500,
)

In [ ]:
line_img.nbytes / 1e6

In [ ]:
line_img[0].plot()

In [ ]:
line_img[-1].plot()

# Sample

In [ ]:
# nd2 = nd2reader.ND2Reader("ClpP_mEGFP_100x.nd2")
# nd2._parser._raw_metadata.z_levels = [0]
# sample_img = nd2.get_frame_2D(v=0, c=1)  # [:,500:1000]
# sample_img = image_to_xarray(sample_img, nd2.metadata["pixel_microns"])

In [ ]:
sample_img = tifffile.imread("t000001xy14c2.tif")
scale = 4.25 / 20
sample_img = sample_img / sample_img.max()
sample_img = simulation.image_to_xarray(sample_img, scale).astype(np.float32)

In [ ]:
labels = sample_img.values > skimage.filters.threshold_otsu(sample_img.values)

In [ ]:
num_spectra = 4

In [ ]:
fp_idx = skimage.morphology.label(labels) % num_spectra + (labels != 0)
for i in range(3):
    fp_idx = skimage.morphology.dilation(fp_idx)
fp_idx = fp_idx.astype(np.uint8)

In [ ]:
fp_img = simulation.xarray_like(sample_img, fp_idx)

# Regrid FP image to excitation line grid

In [ ]:
shifts = dict(x=500, y=470)

In [ ]:
fp_img_offset = simulation.shift_and_interp(
    fp_img, line_img, shifts, method="nearest"
).astype(np.uint8)

In [ ]:
fp_img_offset.plot(aspect=20, size=5)

In [ ]:
sample_img_offset = simulation.shift_and_interp(sample_img, line_img, shifts).astype(
    np.float32
)

In [ ]:
(fp_img_offset * sample_img_offset).plot(aspect=20, size=5)

# Laser spectrum

In [ ]:
laser_spectrum = (
    scipy.stats.norm.pdf(excitation_bins, 550, 10).astype(np.float32)
    * excitation_bin_size
)
laser_spectrum = xr.DataArray(
    laser_spectrum, coords=dict(ex=excitation_bins), dims=["ex"]
)

In [ ]:
laser_spectrum.plot()

# Laser filter spectrum

# Excitation image

In [ ]:
excitation_img = line_img * laser_spectrum

# Absorption image

In [ ]:
fp_img_offset

In [ ]:
sample_excitation_img = xr.DataArray(
    np.rollaxis(fp_excitation_spectra[fp_img_offset, :].values, -1),
    coords=dict(ex=excitation_bins, x=fp_img_offset.x, y=fp_img_offset.y),
    dims=["ex", "y", "x"],
)

In [ ]:
sample_excitation_intensity_img = (sample_excitation_img * sample_img_offset).fillna(0)

In [ ]:
sample_excitation_intensity_img.dtype

In [ ]:
sample_excitation_intensity_img[30].plot(aspect=20, size=5)

In [ ]:
sample_excitation_intensity_img[:, 30, 3180:3230].plot(aspect=20, size=5)

In [ ]:
absorption_img = excitation_img * sample_excitation_intensity_img

In [ ]:
absorption_img[60].plot(aspect=20, size=5)

In [ ]:
absorption_img.dtype

In [ ]:
fp_idx.shape

In [ ]:
fp_emission_spectra.shape

# Emission image

In [ ]:
@numba.guvectorize(
    "(float32[:,:,:,:], uint8[:,:], float32[:,:,:], float32[:,:,:])",
    "(ex,y,x,k), (y,x), (n,ex,em) -> (em,y,x)",
)
def _generate_emission(absorption_img, idx_img, emission_spectra, out):
    for y in range(absorption_img.shape[1]):
        for x in range(absorption_img.shape[2]):
            out[:, y, x] = (
                absorption_img[:, y, x, :] * emission_spectra[idx_img[y, x], :, :]
            ).sum(axis=0)
            # out[:,y,x] = (absorption_img[:,y,x][:,np.newaxis] * emission_spectra[idx_img[y,x],:,:]).sum(axis=0)


def generate_emission(absorption_img, idx_img, emission_spectra):
    ary = _generate_emission(
        np.expand_dims(absorption_img.values, -1),
        idx_img.values,
        emission_spectra.values,
    )
    return xr.DataArray(
        ary,
        coords=dict(em=emission_spectra.em, y=absorption_img.y, x=absorption_img.x),
        dims=["em", "y", "x"],
    )

In [ ]:
%%time
emission_img = generate_emission(absorption_img, fp_img_offset, fp_emission_spectra)

In [ ]:
emission_img[:, 30].plot(aspect=20, size=5)

# Reflected excitation

In [ ]:
# rebin excitation_img according to emission_img

In [ ]:
reflected_excitation_img = excitation_img.interp(ex=emission_bins).rename(ex="em")

In [ ]:
coverslip_reflectivity = 0.1
emission_with_reflected = (
    emission_img + coverslip_reflectivity * reflected_excitation_img
)

In [ ]:
emission_with_reflected[130].plot()

# Resampling to objective resolution

In [ ]:
# 2D blur to NA
# resample x coord (blur to NA, bin+mean [do we need to be careful about normalization?])
# blur y coord to NA
# we have camera y,x grid; groupby_bins each em-slice to the camera y,x grid
# do this fast??
# multiply by camera QE (along em)
# sum along em

In [ ]:
emission_blurred = emission_with_reflected.copy(deep=True)

In [ ]:
scale_x = simulation.bin_size(emission_with_reflected.x.values)
scale_y = simulation.bin_size(emission_with_reflected.y.values)

In [ ]:
objective_na = 0.8

In [ ]:
emission_blurred[130].plot(aspect=20, size=5)

In [ ]:
airy_sigma = 0.42 * 400 * 1e-3 * (2 / objective_na)
i = 130
x = scipy.ndimage.gaussian_filter(
    emission_blurred.values[i],
    (airy_sigma / scale_y, airy_sigma / scale_x),
    output=emission_blurred.values[i],
)

In [ ]:
emission_blurred[131].plot(aspect=20, size=5)

In [ ]:
for i in trange(emission_blurred.sizes["em"]):
    airy_sigma = 0.42 * emission_blurred.em.values[i] * 1e-3 * (2 / objective_na)
    scipy.ndimage.gaussian_filter(
        emission_blurred.values[i],
        (airy_sigma / scale_y, airy_sigma / scale_x),
        output=emission_blurred.values[i],
    )

In [ ]:
emission_blurred[131].plot(aspect=20, size=5)

# Prism

In [ ]:
zero_deflection = 800  # nm
max_deflection = 300  # nm
sensor_spectrum_height = 100  # px

In [ ]:
sensor_y_bins = np.arange(0, sensor_spectrum_height + 1)
# TODO: need to adjust endpoints to match zero/max deflection
# 0.5 shifts (first pixel is 0-to-1)
prism_displacements = (
    np.linspace(0, sensor_spectrum_height, emission_blurred.sizes["em"])[::-1] + 0.5
)

In [ ]:
prism_output = [
    simulation.shift_xarray(emission_blurred[i].copy(), {"y": prism_displacements[i]})
    for i in range(emission_blurred.sizes["em"])
]

In [ ]:
prism_output[151].plot()

In [ ]:
# TODO: rename+relabel y_bins to midpoint (?)
prism_output_binned = [
    em_slice.groupby_bins("y", sensor_y_bins).mean().fillna(0)
    for em_slice in tqdm(prism_output)
]
prism_output_binned = xr.concat(prism_output_binned, dim="em", join="exact")

In [ ]:
prism_output_binned[80].plot()

# Camera

In [ ]:
camera_incident_no_qe = prism_output_binned.sum("em")

In [ ]:
camera_incident_no_qe.plot(size=20)

# Image splitting+Kinetix vs. IMX342

## Thorlabs mirrors/lenses

In [ ]:
mirror_spectra = {
    "UV": "data/thorlabs_mirrors/Thorlabs_UV-Enhanced_Aluminum_Coating.xlsx",
    "Aluminum": "data/thorlabs_mirrors/Thorlabs_Protected_Aluminum_Coating.xlsx",
    "Silver": "data/thorlabs_mirrors/Thorlabs_Protected_Silver_Coating.xlsx",
    "E01": "data/thorlabs_mirrors/E01ReflectionData.xlsx",
    "E02": "data/thorlabs_mirrors/E02ReflectionData.xlsx",
}
mirror_spectra = pd.DataFrame(
    {
        name: simulation.read_thorlabs(filename)[("45°", "Unpol.")]
        for name, filename in mirror_spectra.items()
    }
).interpolate()

In [ ]:
mirror_spectra.plot(xlim=(300, 800))

In [ ]:
lens_spectra = {
    "A": "data/thorlabs_lenses/A_Broadband_AR-Coating.xlsx",
    "AB": "data/thorlabs_lenses/AB_Broadband_AR-Coating.xlsx",
    "UVFS": "data/thorlabs_lenses/UVFS_UV_Broadband_AR-Coating.xlsx",
}
lens_spectra = pd.DataFrame(
    {
        name: simulation.read_thorlabs(filename)["Reflectance"]
        for name, filename in lens_spectra.items()
    }
).interpolate()

In [ ]:
lens_spectra.plot(xlim=(300, 800))

## Sensor QE

In [ ]:
imx342 = pd.read_csv(
    "data/sensor_qe/imx342.csv", names=["Wavelength", "QE"], skiprows=1
)
# imx342.columns = ["Wavelength", "QE"]
imx342.set_index("Wavelength", inplace=True)
imx342 /= 100

In [ ]:
kinetix = pd.read_csv("data/sensor_qe/kinetix.csv", names=["Wavelength", "QE"])
kinetix.set_index("Wavelength", inplace=True)
kinetix /= 100

## Comparison

In [ ]:
emission_path_spectra = pd.concat(
    (
        kinetix["QE"].rename("Kinetix"),
        imx342["QE"].rename("IMX342"),
        mirror_spectra["E02"],
        mirror_spectra["Silver"],
        lens_spectra["A"],
    ),
    axis=1,
).interpolate()
emission_path_spectra.plot(xlim=(300, 800))

In [ ]:
emission_path_spectra["Kinetix final"] = (
    (1 - emission_path_spectra["A"]) ** 2
    * emission_path_spectra["E02"] ** 6
    * emission_path_spectra["Silver"]
    * emission_path_spectra["Kinetix"]
)
emission_path_spectra["IMX342 final"] = (
    1 - emission_path_spectra["A"]
) * emission_path_spectra["IMX342"]
emission_path_spectra["Kinetix advantage"] = (
    emission_path_spectra["Kinetix final"] - emission_path_spectra["IMX342 final"]
)

In [ ]:
emission_path_spectra[["Kinetix final", "IMX342 final"]].plot(
    xlim=(300, 800), figsize=(15, 6)
)

In [ ]:
emission_path_spectra["Kinetix advantage"].plot(xlim=(300, 800), figsize=(15, 6))

# Laser tuning curve

## NKT

In [ ]:
nkt = pd.read_excel("data/laser_tuning/FIU-15.xlsx", columns=["Power density"])
nkt.set_index("Wavelength (nm)", inplace=True)
nkt = nkt[~nkt.index.duplicated(keep="first") & ~nkt.index.isnull()]
nkt = simulation.interpolate_dataframe(nkt, excitation_bins)

In [ ]:
nkt_5nm = nkt.rolling(window=5).sum().fillna(method="ffill")
nkt_5nm.columns = ["Power"]
hv.Curve(nkt_5nm).opts(aspect=2)

## Coherent Chameleon Discovery+VUE

In [ ]:
discovery_shg = pd.read_csv(
    "data/laser_tuning/discovery_vue_shg.csv", header=1
).set_index("lambda")
# discovery_shg.columns = ["power"]
discovery_pump = pd.read_csv(
    "data/laser_tuning/discovery_vue_pump.csv", header=1
).set_index("lambda")
discovery_pump.columns = ["Pump power"]
discovery = pd.concat(
    [
        discovery_shg,
        discovery_pump,
        discovery_shg.rename(columns={"SHG power": "Power"}).append(
            discovery_pump.rename(columns={"Pump power": "Power"})
        ),
    ],
    axis=1,
)
discovery.index.name = "Wavelength"
discovery = discovery[~discovery.index.duplicated(keep="last")]
discovery = simulation.interpolate_dataframe(discovery, excitation_bins)
hv.Curve(discovery, "Wavelength", "Power").opts(width=800)

# Objectives

In [ ]:
objective = pd.read_csv(
    "data/objective_transmittance/uplxapo_20x.csv",
    header=None,
    names=["Wavelength", "Transmittance"],
).set_index("Wavelength")
objective /= 100
objective = simulation.interpolate_dataframe(objective, excitation_bins)

In [ ]:
hv.Curve(objective).opts(aspect=2)

# Excitation path

In [ ]:
excitation_path_spectra = {
    ("NKT", "UPLXAPO_20x"): objective["Transmittance"] * nkt_5nm["Power"],
    ("Discovery", "UPLXAPO_20x"): objective["Transmittance"] * discovery["Power"],
}
excitation_path_spectra = pd.concat(excitation_path_spectra, axis=1)

In [ ]:
np.log10(excitation_path_spectra).plot()

# FPbase

In [ ]:
%%time
fpbase = requests.get("https://www.fpbase.org/api/proteins/?format=json").json()
fpbase_spectra = requests.get(
    "https://www.fpbase.org/api/proteins/spectra/?format=json"
).json()

In [ ]:
def _spectrum_to_series(x):
    ary = np.array(x)
    return pd.DataFrame(
        ary[:, 1], index=pd.Index(ary[:, 0], name="Wavelength"), columns=["Spectrum"]
    )


fps = {}
for fp in fpbase_spectra:
    fp["spectra"] = {
        s["state"]: update_in(dissoc(s, "state"), ["data"], _spectrum_to_series)
        for s in fp["spectra"]
    }
for db in (fpbase, fpbase_spectra):
    for fp in db:
        if fp["slug"] not in fps:
            fps[fp["slug"]] = {}
        fps[fp["slug"]].update(fp)

In [ ]:
fps["avgfp"]

In [ ]:
len(fpbase_spectra)

In [ ]:
len(fpbase)

In [ ]:
set([fp["switch_type"] for fp in fpbase])

In [ ]:
a = fps["avgfp"]["spectra"]["default_em"]["data"]

In [ ]:
plt.plot(fps["avgfp"]["spectra"]["default_em"]["data"])

In [ ]:
fps["avgfp"]["states"]

In [ ]:
selected_fps = {
    fp["slug"]: fp
    for fp in fps.values()
    if fp["switch_type"] == "b"
    and "states" in fp
    and len(fp["states"]) == 1
    and fp["states"][0]["maturation"]
    #    and fp["states"][0]["maturation"] > 60
    and fp["states"][0]["brightness"]
    and fp["states"][0]["brightness"] / fp["states"][0]["maturation"] > 0.9
    and "spectra" in fp
    #     and fp["states"][0]["ext_coeff"]
    #     and fp["states"][0]["qy"]
    #     and fp["states"][0]["ext_coeff"] * fp["states"][0]["qy"] / 1e3 > 30
    # and "spectra" in fp
    # and "default_ex" in fp["spectra"]
    # and "default_em" in fp
    # and fp["spectra"]["default_ex"]["ec"] * fp["spectra"]["default_em"]["qy"] > 10
}
len(selected_fps)

In [ ]:
a = emission_path_spectra["Kinetix final"]

In [ ]:
pd.core.indexes.base.Index

In [ ]:
pd.MultiIndex.__bases__

In [ ]:
fps["avgfp"]["spectra"]["default_em"]["data"]

In [ ]:
(
    simulation.interpolate_dataframe(
        fps["avgfp"]["spectra"]["default_em"]["data"], emission_path_spectrum.index
    )
)

In [ ]:
(
    simulation.interpolate_dataframe(
        fps["avgfp"]["spectra"]["default_em"]["data"], emission_path_spectrum.index
    )
    * emission_path_spectrum
).sum()

In [ ]:
fps["avgfp"]["states"][0]["ex_max"]

In [ ]:
def get_df_nearest(df, val):
    return df.iloc[df.index.get_loc(val, method="nearest")]

In [ ]:
emission_path_spectrum.truncate(before=395)

In [ ]:
fps["avgfp"]["spectra"]["default_em"]["data"] * a

In [ ]:
fps["avgfp"]["spectra"]["default_em"]["data"].interpolate

In [ ]:
get_df_nearest(excitation_path_spectrum["Transmittance"], 300)

In [ ]:
excitation_path_spectrum = excitation_path_spectra[("NKT", "UPLXAPO_20x")]
emission_path_spectrum = emission_path_spectra["Kinetix final"]
fp_table = []
for fp in selected_fps.values():
    em_efficiency = (
        simulation.interpolate_dataframe(
            fp["spectra"]["default_em"]["data"], emission_path_spectrum.index
        )
        * emission_path_spectrum
    ).sum()
    ex_peak_efficiency = get_df_nearest(
        excitation_path_spectrum, fp["states"][0]["ex_max"]
    )
    brightness = fp["states"][0]["brightness"]
    maturation = fp["states"][0]["maturation"]
    em_brightness = brightness * em_efficiency
    emmat_brightness = em_brightness / maturation
    exem_brightness = brightness * em_efficiency * ex_peak_efficiency
    exemmat_brightness = exem_brightness / maturation
    for key in ["spectra", "states"]:
        fp = dissoc(fp, key)
    fp = {
        "name": fp["name"],
        "brightness": brightness,
        "maturation": maturation,
        "em_efficiency": em_efficiency,
        "ex_peak_efficiency": ex_peak_efficiency,
        "em_brightness": em_brightness,
        "emmat_brightness": emmat_brightness,
        "exem_brightness": exem_brightness,
        "exemmat_brightness": exemmat_brightness,
        **dissoc(fp, "name"),
    }
    fp_table.append(fp)

fp_table = pd.DataFrame(fp_table)
fp_table.sort_values("emmat_brightness", ascending=False).style.set_precision(
    2
).background_gradient(cmap="RdPu", low=0.7, high=0, subset=fp_table.columns[1:9])

In [ ]:
selected_fps.keys()

In [ ]:
25000 * 0.79 / 1e3

In [ ]:
selected_fps